METHODS

Dynamical equations for cell 5:

$N_5(t+1)=[N_5(t)+(N_{2,4,6,8}(t)-4N_5(t))\cdot c]\cdot (1+m_5(t))-H_5(t)-S_5(t)$

$H_5(t)=N_5(t-t_h)\cdot m_5(t-t_h)\cdot h\cdot (1-4c)^{t_h}+N_{2,4,6,8}(t-t_h)\cdot m_5(t-t_h)\cdot h\cdot c\cdot \frac{1-(1-4c)^{t_h}}{1-(1-4c)}$

$S_5(t)=N_5(t-t_s)\cdot m_5(t-t_s)\cdot s\cdot (1-4c)^{t_s}+N_{2,4,6,8}(t-t_s)\cdot m_5(t-t_s)\cdot s\cdot c\cdot \frac{1-(1-4c)^{t_s}}{1-(1-4c)}$

$m_5(t+1)=m\cdot (1-\frac{1}{MAX_5}\cdot [N_5(t)+\sum\limits_{i=1}^t H_5(i)+\sum\limits_{i=1}^t S_5(i)])$

After control:

$N_5(t+1)=\{[N_5(t)+(N_{2,4,6,8}(t)-4N_5(t))\cdot c_{control}]\cdot (1+m_5(t))-H_5(t)-S_5(t)\}\cdot (1-k_{control})$

$H_5(t)=N_5(t-t_h)\cdot m_5(t-t_h)\cdot h\cdot (1-c_{control})^{t_h}+N_{2,4,6,8}(t-t_h)\cdot m_5(t-t_h)\cdot h\cdot c_{control}\cdot \frac{1-(1-c_{control})^{t_h}}{1-(1-c_{control})}$

$S_5(t)=N_5(t-t_s)\cdot m_5(t-t_s)\cdot s\cdot (1-c_{control})^{t_s}+N_{2,4,6,8}(t-t_s)\cdot m_5(t-t_s)\cdot s\cdot c_{control}\cdot \frac{1-(1-c_{control})^{t_s}}{1-(1-c_{control})}$

$m_5(t+1)=min\{m\cdot (1-\frac{1}{MAX_5}\cdot [N_5(t)+\sum\limits_{i=1}^t H_5(i)+\sum\limits_{i=1}^t S_5(i)]), m_{control}\}$


Image processing:

In [1]:
"""
A Cellular automaton simulation of coronavirus cov-19
Created in Apr 2020
parameters need to be modified are highlighted with
##############################
M_INI, C, H, S, TIME_H, TIME_S
M_CONTR, C_CONTR, K_CONTR
##############################
############
time_control
time_total
############
"""


import sys
import random
import numpy as np
import pandas as pd
from numpy import array
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from scipy import signal
# import matplotlib._png as png
from sklearn.cluster import KMeans
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import downscale_local_mean

%matplotlib inline
# np.set_printoptions(threshold=sys.maxsize)


class Cov():
    """
    this is the model for coronavirus cov-19
    """


    def __init__(self, initial, bi_initial, ini_edge, max_matrix, m, c, m_control, c_control, k):


        """
        modify your parameters over here:
        m, c, h, s, m_control, c_control, k_control...
        """
        ####################################################
        # initial spreading coefficient
        self.M_INI = m
        # self.M_INI = 0.4
        # travel parameter, with area of 15km^2 for one cell
        self.C = c
        # self.C = 0.1
        # percentage to hospital
        self.H = 0.2
        # percentage to selfheal
        self.S = 0.8
        # governmental controls
        self.M_CONTR = m_control
        self.C_CONTR = c_control
        self.K_CONTR = k
        # self.M_CONTR = 0.4
        # self.C_CONTR = 0.1
        # self.K_CONTR = 0
        # time period of latency(days)
        self.TIME_H = 6
        # time period of selfheal(days)
        self.TIME_S = 15
        ####################################################

        """
        initial conditions, no adjustment required
        """
        # initialize time
        self.t = 0
        # maximum population of a cell
        self.MAX = max_matrix
        # initialize the map
        self.state = initial
        self.initial = initial
        self.bi_cmap = bi_initial
        self.edge = ini_edge
        self.m = np.full_like(self.state, self.M_INI)
        # initialize the sum of specific population
        self.redu_h = np.full_like(self.state, 0)
        self.redu_s = np.full_like(self.state, 0)
        self.sum_h = np.full_like(self.state, 0)
        self.sum_s = np.full_like(self.state, 0)
        self.total = np.full_like(self.state, 0)
        # set up neighbor convolution kernel
        """
        convolution kernel used for short-distance migration: 
        assume up, down, left and right have more weights than the other four directions
        0.0 1.0 0.0
        1.0 0.0 1.0
        0.0 1.0 0.0
        """
        self.ker = np.array([[0,1,0],[1,0,1],[0,1,0]])
        # self.ker_sum = np.sum(self.ker)
        self.rows = np.array([], dtype = np.str).reshape(0, len(column_names))
        self.rows_h = np.array([], dtype = np.str).reshape(0, len(column_names))
        self.rows_total = np.array([], dtype = np.str).reshape(0, len(column_names))


    def count_neighb(self):
        """
        use 2D convolution to calculate neighbor weight counts
        """
        neigh_count = signal.convolve2d(self.bi_cmap, self.ker, 'same', boundary = 'fill')
        return neigh_count


    def count_neighb_popu(self, state):
        """
        use 2D convolution to calculate neighbor populations
        """
        popu_count = signal.convolve2d(state, self.ker, 'same', boundary = 'fill')
        return popu_count


    def upadate_m(self):
        """
        update m at each step according to the rules of immunity
        """
        neighb_count = self.count_neighb()+0.001
        #initial_modified = 0
        initial_modified = (self.initial * (1-neighb_count*self.C)**self.t
            + self.count_neighb_popu(self.initial) * self.C * ((1-(1-neighb_count*self.C)**self.t)/(1-(1-neighb_count*self.C))))
        self.total = self.state + self.sum_s + self.sum_h + initial_modified
        self.m = self.M_INI * (1 - self.total/self.MAX)
        # print(self.m[np.where(pi == pi[209][178])])


    def spread(self):
        """
        natural spreading according to the R0 rate and short-distance migration
        """
        neighb = self.count_neighb_popu(self.state)
        num = (self.state + (neighb-self.count_neighb()*self.state)*self.C) * (1 + self.m)
        return num


    def hospital(self):
        """
        population isolated by hospitals
        """
        neighb_count = self.count_neighb()+0.001
        preneighb = self.count_neighb_popu(self.prestate)
        self.redu_h = (self.prestate * self.pre_m * self.H * (1-neighb_count*self.C)**self.TIME_H
            + preneighb * self.pre_m * self.H * self.C * ((1-(1-neighb_count*self.C)**self.TIME_H)/(1-(1-neighb_count*self.C))))
        self.sum_h += self.redu_h
        return self.redu_h


    def selfheal(self):
        """
        population that heal themselves outside of hospitals
        """
        neighb_count = self.count_neighb()+0.001
        preneighb2 = self.count_neighb_popu(self.prestate2)
        self.redu_s = (self.prestate2 * self.pre_m2 * self.S * (1-neighb_count*self.C)**self.TIME_S
            + preneighb2 * self.pre_m2 * self.S * self.C * ((1-(1-neighb_count*self.C)**self.TIME_S)/(1-(1-neighb_count*self.C))))
        self.sum_s += self.redu_s
        return self.redu_s


    def update_early(self):
        """
        early stage, all patients are in the incubation period 
        """
        self.upadate_m()
        self.state = self.spread()
        self.state[self.edge] = 0
        self.state[np.where(self.state < 0)] = 0


    def update_found(self):
        """
        after a period of 10 days
        first case of being tested positive(hospital isolation) appears
        """
        self.upadate_m()
        self.state = self.spread() - self.hospital()
        self.state[self.edge] = 0
        self.state[np.where(self.state < 0)] = 0


    def update_selfheal(self):
        """
        after a period of 25 days
        first case of selfheal appears
        """
        self.upadate_m()
        self.state = self.spread() - self.hospital() - self.selfheal()
        self.state[self.edge] = 0
        self.state[np.where(self.state < 0)] = 0


    def update_control(self):
        """
        introduce governmental control
        """
        self.upadate_m()
        self.m = np.minimum(self.m, np.full_like(self.m, self.M_CONTR))
        self.C = self.C_CONTR
        state_bef_contr = self.spread() - self.hospital() - self.selfheal()
        self.state = state_bef_contr * (1 - self.K_CONTR)
        self.state[self.edge] = 0
        self.state[np.where(self.state < 0)] = 0


    def show_numcases(self):
        """
        show number of cases of every regions
        """
        tmp = 0
        
        totalnum = 0
        totalnum_h = 0
        totalnum_total =0
        
        ans = []
        
        new_row = np.array([])
        new_row_h = np.array([])
        new_row_total = np.array([])
        
        for key, region in regions.items():
            
            numcase = int(self.state[region["loc"]].sum())
            numcase_h = int(self.redu_h[region["loc"]].sum())
            numcase_total = int(self.total[region["loc"]].sum())
            
            new_row = np.append(new_row, numcase)
            new_row_h = np.append(new_row_h, numcase_h)
            new_row_total = np.append(new_row_total, numcase_total)
            
            totalnum += numcase
            totalnum_h += numcase_h
            totalnum_total += numcase_total
            
            name = region["name"]
            an = plt.annotate("{}:{}".format(name, numcase), xy=(200, 25 + tmp*10))
            ans.append(an)
            tmp += 1
        name = "Total"
        
        numcase = totalnum
        numcase_h = totalnum_h
        numcase_total = totalnum_total
        
        new_row = np.append(new_row, numcase)
        new_row_h = np.append(new_row_h, numcase_h)
        new_row_total = np.append(new_row_total, numcase_total)
        
        an = plt.annotate("{}:{}".format(name, numcase), xy=(200, 25 + tmp*10))
        ans.append(an)
        self.rows = np.vstack((self.rows, new_row))
        self.rows_h = np.vstack((self.rows_h, new_row_h))
        self.rows_total = np.vstack((self.rows_total, new_row_total))
        return ans


    def simulate(self, num_con, num_gen):
        """
        run the simulation by time steps
        """
        fig = plt.figure(figsize = (10,10))
        plt.axis("off")
        states = []
        ms =[]
        ims = []
        cmax = np.log(np.amax(self.MAX)+1)
        # print(np.amax(self.MAX))
        cnorm = mpl.colors.Normalize(vmin=0,vmax=cmax)
        im_edg = plt.imshow(self.bi_cmap, cmap='gray')
        ccmap=mpl.cm.Reds
        ccmap.set_bad(color='aqua')
        
        for i in range(self.TIME_H):
            self.t = i+1
            if np.any(np.log(self.state+1) > cmax):
                break
            if np.any(np.log(self.state+1) < 0):
                print("error: negative numbers occur")
                break
            # print("{}:{}".format(i, np.sum(self.state)))
            # print("{}:{}".format(i, self.state[5][5]))
            states.append(self.state)
            ms.append(self.m)
            im_value = np.log(self.state+1)
            im_value[self.edge] = np.NAN
            # print(len(im_value[np.where(im_value<0)]))
            im = plt.imshow(im_value, cmap=ccmap, norm = cnorm, animated = True)           
            tm = plt.annotate("DAY {}".format(i+1), xy=(25,25))
            ims.append([im, tm])
            # ims.append([im, tm] + self.show_numcases())
            self.update_early()

        for i in range(self.TIME_H, self.TIME_S):
            self.t = i+1
            if np.any(np.log(self.state+1) > cmax):
                break
            if np.any(np.log(self.state+1) < 0):
                print("error: negative numbers occur")
                break
            # print("{}:{}".format(i, np.sum(self.state)))
            # print("{}:{}".format(i, self.state[5][5]))
            states.append(self.state)
            ms.append(self.m)
            # 10 days before
            self.prestate = states[i-self.TIME_H]
            self.pre_m = ms[i-self.TIME_H]
            im_value = np.log(self.state+1)
            im_value[self.edge] = np.NAN
            # print(len(im_value[np.where(im_value<0)]))
            im = plt.imshow(im_value, cmap=ccmap, norm = cnorm, animated = True)
            tm = plt.annotate("DAY {}".format(i+1), xy=(25,25))
            ims.append([im, tm])
            # ims.append([im, tm] + self.show_numcases())
            self.update_found()

        #remove initial numbers
#         neighb_count = self.count_neighb()+0.001
#         initial_modified_ts = (self.initial * (1-neighb_count*self.C)**self.TIME_S
#             + self.count_neighb_popu(self.initial) * self.C * ((1-(1-neighb_count*self.C)**self.TIME_S)/(1-(1-neighb_count*self.C))))
#         self.state = self.state - initial_modified_ts

        for i in range(self.TIME_S, num_con):
            self.t = i+1
            if np.any(np.log(self.state+1) > cmax):
                break
            if np.any(np.log(self.state+1) < 0):
                print("error: negative numbers occur")
                break
            # print("{}:{}".format(i, np.sum(self.state)))
            # print("{}:{}".format(i, self.state[5][5]))
            states.append(self.state)
            ms.append(self.m)
            # 10 days before
            self.prestate = states[i-self.TIME_H]
            self.pre_m = ms[i-self.TIME_H]
            # 25 days before
            self.prestate2 = states[i-self.TIME_S]
            self.pre_m2 = ms[i-self.TIME_S]
            im_value = np.log(self.state+1)
            im_value[self.edge] = np.NAN
            # print(len(im_value[np.where(im_value<0)]))
            im = plt.imshow(im_value, cmap=ccmap, norm = cnorm, animated = True)
            tm = plt.annotate("DAY {}".format(i+1), xy=(25,25))
            ims.append([im, tm])
            # ims.append([im, tm] + self.show_numcases())
            self.update_selfheal()

        for i in range(num_con, num_gen):
            self.t = i+1
            if np.any(np.log(self.state+1) > cmax):
                break
            if np.any(np.log(self.state+1) < 0):
                print("error: negative numbers occur")
                break
            # print("{}:{}".format(i, np.sum(self.state)))
            # print("{}:{}".format(i, self.state[5][5]))
            states.append(self.state)
            ms.append(self.m)
            # 10 days before
            self.prestate = states[i-self.TIME_H]
            self.pre_m = ms[i-self.TIME_H]
            # 25 days before
            self.prestate2 = states[i-self.TIME_S]
            self.pre_m2 = ms[i-self.TIME_S]
            im_value = np.log(self.state+1)
            im_value[self.edge] = np.NAN
            # print(len(im_value[np.where(im_value<0)]))
            im = plt.imshow(im_value, cmap=ccmap, norm = cnorm, animated = True)
            tm = plt.annotate("DAY {}".format(i+1), xy=(25,25))
            ims.append([im, tm])
            # ims.append([im, tm] + self.show_numcases())
            self.update_control()
        
        # fig.colorbar(im, extend='both')
        
        # np.savetxt("state40",states[40])
        anim = animation.ArtistAnimation(
            fig, ims , interval=500, repeat=False, blit=True
        )
        plt.close()
        return anim, self.rows, self.rows_h, self.rows_total


"""
this section is for image processing (map of UK)
"""


try:
    pi = io.imread('postprocess_ukmap.png')
    # pi = png.read_png_int('postprocess_ukmap.png')
except:
    print("image is not ready, performing image processing...")
    # url = "https://github.com/daweiliucsd/Cov19-model/blob/master/england.png"
    # pi = imread(url)
    pi = png.read_png_int('ukmap.png')
    # pi = io.imread('ukmap.png', 'png')

    print(pi.shape)
    print("white:", pi[0][0])
    # print(pi[0][0].dtype)
    white = pi[0][0]
    colors=[]
    print("purple:", pi[98][116])
    colors.append(pi[98][116])
    print("red1:", pi[190][190])
    colors.append(pi[190][190])
    print("red2:", pi[138][156])
    colors.append(pi[138][156])
    print("pink:", pi[215][164])
    colors.append(pi[215][164])
    print("orange:", pi[183][140])
    colors.append(pi[183][140])
    print("London:", pi[209][178])
    colors.append(pi[209][178])
    print("yellow:", pi[108][145])
    colors.append(pi[108][145])
    print("green1:", pi[123][132])
    colors.append(pi[123][132])
    print("green2:", pi[173][163])
    colors.append(pi[173][163])
    print("blue1:", pi[219][136])
    colors.append(pi[219][136])
    print("blue2:", pi[189][116])
    colors.append(pi[189][116])
    print("Ireland:", pi[120][75])
    colors.append(pi[120][75])
    # print("island:", pi[129][102])
    # print(colors[0].dtype)


    bi_color_pi=np.zeros((pi.shape[0],pi.shape[1]))
    for i, row in enumerate(pi):
        for j, element in enumerate(row):
            if any((element == color).all() for color in colors):
                bi_color_pi[i][j] = 1
            else:
                bi_color_pi[i][j] = 0


    def nearest_nonzero_idx(a,x,y):
        idx = np.argwhere(a)
        idx_i = idx[((idx - [x,y])**2).sum(1).argmin()][0]
        idx_j = idx[((idx - [x,y])**2).sum(1).argmin()][1]
        return idx_i, idx_j


    for i, row in enumerate(pi):
        for j, element in enumerate(row):
            if any((element == color).all() for color in colors):
                continue
            elif (element == white).all():
                continue
            else: 
                print(nearest_nonzero_idx(bi_color_pi, i, j))
                pi[i][j] = pi[nearest_nonzero_idx(bi_color_pi, i, j)[0]][nearest_nonzero_idx(bi_color_pi, i, j)[1]]


    plt.imshow(pi)
    io.imsave('postprocess_ukmap.png', pi)

    # RGB = pi.reshape(-1, pi.shape[2])
    # print(RGB.shape)
    # for index, cell in enumerate(RGB):
    #     if any((cell == color).all() for color in colors):
    #         continue
    #     else:
    #         RGB[index] = pi[0][0]
    # post_map = RGB.reshape(pi.shape)



    # n_colors = 13
    # arr = pi.reshape(-1, pi.shape[2])
    # kmeans = KMeans(n_clusters=n_colors, random_state=42).fit(arr)
    # labels = kmeans.labels_
    # centers = kmeans.cluster_centers_
    # less_colors = centers[labels].reshape(pi.shape).astype('uint8')

    # io.imshow(less_colors)


"""
initiate situations
online data are saved in a nested-dictionary called regions
pop: population of corresponding region
cases: cov19 cases of corresponding region
loc: index locations of corresponding region
"""


# pi = downscale_local_mean(pi, (10, 10, 1))
pi_3d = io.imread('postprocess_ukmap.png')
# pi_3d = png.read_png_int('postprocess_ukmap.png')
pi = rgb2gray(pi_3d)
# print(pi_gray)
# print(pi.dtype)
# print("image size:", pi.shape)
# print("number of different colors of the original image:", len(np.unique(pi)))
a = pi.shape[0]
b = pi.shape[1]
cmap = np.zeros((a,b))
cmap_concentrate = np.zeros((a,b))
max_matrix = np.zeros((a,b)) + 10000


regions = {
    # east england
    "ea" : {
        "name" : "East England",
        "pop" : 6.2 * 1000000,
        "cases" : 3,
        "loc" : np.where(pi == pi[190][190])
  },
    # london
    "ld" : {
        "name" : "London",
        "pop" : 8.9 * 1000000,
        "cases" : 4,
        "loc" : np.where(pi == pi[209][178])
  },
    # midlands
    "ml" : {
        "name" : "Midlands",
        "pop" : 10.7 * 1000000,
        "cases" : 1,
        "loc" : np.where((pi == pi[183][140]) | (pi == pi[173][163]))
  },
    # yorkshire & north east
    "yn" : {
        "name" : "North East",
        "pop" : 8.14 * 1000000,
        "cases" : 4,
        "loc" : np.where((pi == pi[138][156]) | (pi == pi[108][145]))
  },
    # north west
    "nw" : {
        "name" : "North West",
        "pop" : 7.29 * 1000000,
        "cases" : 1,
        "loc" : np.where(pi == pi[123][132])
  },
    # south east
    "se" : {
        "name" : "South East",
        "pop" : 9.13 * 1000000,
        "cases" : 10,
        "loc" : np.where(pi == pi[215][164])
  },
    # south west
    "sw" : {
        "name" : "South West",
        "pop" : 5.56 * 1000000,
        "cases" : 3,
        "loc" : np.where(pi == pi[219][136])
  },
    # north ireland
    "ni" : {
        "name" : "North Ireland",
        "pop" : 1.88 * 1000000,
        "cases" : 3,
        "loc" : np.where(pi == pi[120][75])
  },
    # scotland
    "sl" : {
        "name" : "Scotland",
        "pop" : 5.44 * 1000000,
        "cases" : 3,
        "loc" : np.where(pi == pi[98][116])
  },
    # wales
    "wl" : {
        "name" : "Wales",
        "pop" : 3.13 * 1000000,
        "cases" : 1,
        "loc" : np.where(pi == pi[189][116])
  }
}

############################################################################
# version 1: different regions have different max cellular population limit

pseudo_random = [619, 64, 979, 794, 474, 634, 782, 414, 2646, 683]
num_total = 0
i = 0
column_names = []
for key, region in regions.items():
    name = region["name"]
    column_names.append(name)
    num_pi = len(region["loc"][0])
    num_total += num_pi
    # idx = random.randint(0, num_pi-1)
    idx = pseudo_random[i]
    i += 1
    cmap_concentrate[tuple(x[idx] for x in region["loc"])] = region["cases"]
    cmap[region["loc"]] = region["cases"]/num_pi
    max_matrix[region["loc"]] = region["pop"]/num_pi
column_names.append('All')

bi_cmap = np.zeros((a,b))
bi_cmap[np.where(cmap == 0)] = 0
bi_cmap[np.where(cmap != 0)] = 1
print("total pixels of land:{}".format(np.sum(bi_cmap)))
############################################################################


# ############################################################################
# # version 2: different regions have same max cellular population limit

# pseudo_random = [619, 64, 979, 794, 474, 634, 782, 414, 2646, 683]
# num_total = 0
# i = 0
# for key, region in regions.items():
#     name = region["name"]
#     column_names.append(name)
#     num_pi = len(region["loc"][0])
#     num_total += num_pi
#     # idx = random.randint(0, num_pi-1)
#     idx = pseudo_random[i]
#     i += 1
#     cmap_concentrate[tuple(x[idx] for x in region["loc"])] = region["cases"]
#     cmap[region["loc"]] = region["cases"]/num_pi
# column_names.append('All')

# bi_cmap = np.zeros((a,b))
# bi_cmap[np.where(cmap == 0)] = 0
# bi_cmap[np.where(cmap != 0)] = 1
# max_matrix = np.zeros((a,b)) + 66370000/np.sum(bi_cmap)
# ############################################################################


# a = pi.shape[0]
# b = pi.shape[1]
# cmap = np.zeros((a,b))
# bi_cmap = np.zeros((a,b))
# for i in range(a):
#     for j in range(b):
#         if pi[i][j][0] == 255 and pi[i][j][1] == 255 and pi[i][j][2] == 255:
#             cmap[i][j] = -1
#         else:
#             cmap[i][j] = pi[i][j][0]/10
#             bi_cmap[i][j] = 1


"""
generate html video
modify simulation time here
time_total: total days of simulation
time_control: the day to add the goverment controls
"""
###################
# time_control should be larger than TIME_S
time_control = 16
time_total = 100

m = 0.4
c = 0.1
m_control = 0.3
c_control = 0.08
k = 0.05
###################
# print(cmap_concentrate)
init = cmap_concentrate
ocean = np.where(bi_cmap == 0)
Simulation = Cov(init, bi_cmap, ocean, max_matrix, m, c, m_control, c_control, k)
print("Running ......")
anim = Simulation.simulate(time_control, time_total)[0]
# HTML(anim.to_html5_video())


# uncomment the next line if you want to save the video in MP4
anim.save("england.mp4")
# anim.save('animation.gif', writer='imagemagick', fps=10)
print("Finished")


In [3]:
print("Running ......")
for m in [0.8]:
    for c in [0.1]:
        for m_control in [0.3]:
            for c_control in [0.08]:
                for k in [0.05]:
                    Simulation = Cov(init, bi_cmap, ocean, max_matrix, m, c, m_control, c_control, k)
                    anim, rows, rows_h, rows_total = Simulation.simulate(time_control, time_total)
                    df = pd.DataFrame(rows, columns = column_names)
                    df_h = pd.DataFrame(rows_h, columns = column_names)
                    df_total = pd.DataFrame(rows_total, columns = column_names)
                    # df.to_csv('england_data_tables/m{}_c{}_mc{}_cc{}_k{}.csv'.format(m, c, m_control, c_control, k))
                    df.to_csv('daily_hospital_and_total/case_today.csv')
                    df_h.to_csv('daily_hospital_and_total/case_today_into_hospital.csv')
                    df_total.to_csv('daily_hospital_and_total/case_total.csv')
print("Finished")

Running ......


/usr/local/lib/python3.6/dist-packages/ipykernel/__main__.py:281: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("Reds"))


Finished
